<a href="https://colab.research.google.com/github/zbc89631412/zbc89631412.github.io/blob/master/ml_assignment3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install torch torchvision

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
NUM_EPOCH = 10
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
class CifarResNet(nn.Module):

    def __init__(self, block, layers, num_classes=100):
        super(CifarResNet, self).__init__()
        self.inplanes = 16
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
class cifar_resnet20(nn.Module):
    def __init__(self):
        super(cifar_resnet20, self).__init__()
        ResNet20 = CifarResNet(BasicBlock, [3, 3, 3])
        url ='https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar100_resnet20-23dac2f1.pt'
		
        ResNet20.load_state_dict(model_zoo.load_url(url))
        modules = list(ResNet20.children())[:-1]
        backbone = nn.Sequential(*modules)
        self.backbone = nn.Sequential(*modules)
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        out = self.backbone(x)
        out = out.view(out.shape[0], -1)
        return self.fc(out)

In [3]:
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = cifar_resnet20()
    model.to(torch.device("cuda:0"))
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                                         std=(0.2023, 0.1994, 0.2010))])
    trainset = datasets.CIFAR10('./data', download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                              shuffle=True, num_workers=2)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, num_workers=2)
    criterion = nn.CrossEntropyLoss()
    lambdas = [0.01,0.001,0.0001,0.00001,0.000001,0.0000001]
    all_data = [[]]
    for lam in lambdas:
      optimizer = optim.SGD(list(model.fc.parameters()), lr=0.002, momentum=0.8 ,weight_decay = lam)
      testing_loss = 10000.0
      best_acc = 0
      ## Do the training
      for epoch in range(NUM_EPOCH):  # loop over the dataset multiple times
          running_loss = 0.0
          temp_loss = 0.0
          for i, data in enumerate(trainloader, 0):
              # get the inputs
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward + backward + optimize
              outputs = model(inputs)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()
              running_loss += loss.item()
              if i % 20 == 19:  # print every 20 mini-batches
                  print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 20))
                  running_loss = 0.0
          #print('Finished Training')
          correct = 0
          total = 0
          count = 0
          with torch.no_grad():
          
            for i, data in enumerate(testloader, 0):
                # get the inputs
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)

        
                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                temp_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                count += 1
                #progress_bar(i, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (temp_loss/(i+1), 100.*correct/total, correct, total)
            acc = 100. * correct / total  
            #print("count is ",count)
            #print(" acc is ",acc)
            
            temp_loss = temp_loss/count
            #testing_loss = testing_loss/312
            print("current loss is ",temp_loss)
            data = [lam, epoch+1, temp_loss, acc]
            all_data.append(data)
            if temp_loss < testing_loss:
                testing_loss = temp_loss
                best_acc = acc
                #print("best acc is ",best_acc)
                torch.save(model, 'model.pth')        
      print("test loss is", testing_loss)
      print("the best acc for", lam ,"is ",best_acc)

    print(all_data)

Files already downloaded and verified
Files already downloaded and verified
[1,    20] loss: 2.328
[1,    40] loss: 2.133
[1,    60] loss: 1.960
[1,    80] loss: 1.885
[1,   100] loss: 1.738
[1,   120] loss: 1.648
[1,   140] loss: 1.622
[1,   160] loss: 1.556
[1,   180] loss: 1.520
[1,   200] loss: 1.392
[1,   220] loss: 1.449
[1,   240] loss: 1.354
[1,   260] loss: 1.348
[1,   280] loss: 1.345
[1,   300] loss: 1.306
[1,   320] loss: 1.289
[1,   340] loss: 1.319
[1,   360] loss: 1.279
[1,   380] loss: 1.283
[1,   400] loss: 1.213
[1,   420] loss: 1.284
[1,   440] loss: 1.252
[1,   460] loss: 1.243
[1,   480] loss: 1.205
[1,   500] loss: 1.252
[1,   520] loss: 1.233
[1,   540] loss: 1.228
[1,   560] loss: 1.198
[1,   580] loss: 1.155
[1,   600] loss: 1.123
[1,   620] loss: 1.133
[1,   640] loss: 1.139
[1,   660] loss: 1.203
[1,   680] loss: 1.096
[1,   700] loss: 1.158
[1,   720] loss: 1.112
[1,   740] loss: 1.105
[1,   760] loss: 1.155
[1,   780] loss: 1.145
[1,   800] loss: 1.114
[1, 